<img src='otus.png'>

In [2]:
from __future__ import division, print_function
import math, random
from collections import defaultdict, Counter
from scipy import spatial
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Опрос в начале лекции:
https://docs.google.com/forms/d/e/1FAIpQLScoGNFmWENp4nMoxlpdGiALl4hwcVgFHomds8ldJqgUDSRpsQ/viewform?usp=sf_link

# Рекомендательные системы

Заинтересованность компаний в разработке рекомендательных систем:

* покупка некоторого товара
* потребление некоторого контента и удержание пользователя

Покупка товара:

* Amazon
* Ozon
* Lamoda
* ...
* Ваши варианты?

### Lamoda
<img src='lamoda.png'>

### Amazon
<img src='amazon.png'>

Удержание пользователя:
* Spotify
* Netflix
* LinkedIn
* Facebook
* ...
* Ваши варианты?

### LinkedIn

<img src='linkedin.png'>

### LastFM

<img src='lastfm.png'>

Judging by Amazon’s success, the recommendation system works. The company reported a 29% sales increase to \$12.83 billion during its second fiscal quarter, up from $9.9 billion during the same time last year. 

http://fortune.com/2012/07/30/amazons-recommendation-secret/

Уровни работы рекомендательных систем

* Долгосрочные интересы - медленно меняющиеся предпочтения
* Краткосрочные интересы - некоторые кратковременные тренды

Будем рассматривать модели для построения долгосрочных рекомендаций

Рассмотрим пример из книги Data Science from scratch

https://github.com/joelgrus/data-science-from-scratch/

In [4]:
# темы повторяются у разных пользователей

users_interests = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

unique_interests = sorted({interest for user_interests in users_interests for interest in user_interests})

print(unique_interests)
print(len(unique_interests))
print(len(users_interests))

['Big Data', 'C++', 'Cassandra', 'HBase', 'Hadoop', 'Haskell', 'Java', 'Mahout', 'MapReduce', 'MongoDB', 'MySQL', 'NoSQL', 'Postgres', 'Python', 'R', 'Spark', 'Storm', 'artificial intelligence', 'databases', 'decision trees', 'deep learning', 'libsvm', 'machine learning', 'mathematics', 'neural networks', 'numpy', 'pandas', 'probability', 'programming languages', 'regression', 'scikit-learn', 'scipy', 'statistics', 'statsmodels', 'support vector machines', 'theory']
36
15


Если пользователей и тем мало, то справится эксперт, и систему разрабатывать не нужно.  
Когда количество пользователей и тем/товаров исчисляется миллионами, давать рекомендацию нужно автоматически.

## Популярность темы

Простой подход - рекомендуем пользователю самые популярные темы

In [5]:
popular_interests = Counter(
    interest for user_interests in users_interests for interest in user_interests
).most_common()

print("Popular Interests")
print(popular_interests)

Popular Interests
[('Python', 4), ('R', 4), ('Big Data', 3), ('HBase', 3), ('Java', 3), ('statistics', 3), ('regression', 3), ('probability', 3), ('Hadoop', 2), ('Cassandra', 2), ('MongoDB', 2), ('Postgres', 2), ('scikit-learn', 2), ('statsmodels', 2), ('pandas', 2), ('machine learning', 2), ('libsvm', 2), ('C++', 2), ('neural networks', 2), ('deep learning', 2), ('artificial intelligence', 2), ('Spark', 1), ('Storm', 1), ('NoSQL', 1), ('scipy', 1), ('numpy', 1), ('decision trees', 1), ('Haskell', 1), ('programming languages', 1), ('mathematics', 1), ('theory', 1), ('Mahout', 1), ('MapReduce', 1), ('databases', 1), ('MySQL', 1), ('support vector machines', 1)]


In [7]:
def most_popular_new_interests(user_interests, max_results=5):
    suggestions = [(interest, frequency) 
                   for interest, frequency in popular_interests
                   if interest not in user_interests]
    return suggestions[:max_results]


print("Most Popular New Interests")
print("already like:", ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"])
print(most_popular_new_interests(["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"]))

Most Popular New Interests
already like: ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres']
[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]


Такой подход применим для пользователей, о которых ничего не известно. Так называемый "холодный старт".
Почему он плохо работает для всех?

In [8]:
# составим матрицу интересов пользователей

v = DictVectorizer(sparse=False)

d = [{item: 1 for item in ui} for ui in users_interests]
print(d)
user_interest_matrix = v.fit_transform(d)

user_interest_matrix

[{'Hadoop': 1, 'Big Data': 1, 'HBase': 1, 'Java': 1, 'Spark': 1, 'Storm': 1, 'Cassandra': 1}, {'NoSQL': 1, 'MongoDB': 1, 'Cassandra': 1, 'HBase': 1, 'Postgres': 1}, {'Python': 1, 'scikit-learn': 1, 'scipy': 1, 'numpy': 1, 'statsmodels': 1, 'pandas': 1}, {'R': 1, 'Python': 1, 'statistics': 1, 'regression': 1, 'probability': 1}, {'machine learning': 1, 'regression': 1, 'decision trees': 1, 'libsvm': 1}, {'Python': 1, 'R': 1, 'Java': 1, 'C++': 1, 'Haskell': 1, 'programming languages': 1}, {'statistics': 1, 'probability': 1, 'mathematics': 1, 'theory': 1}, {'machine learning': 1, 'scikit-learn': 1, 'Mahout': 1, 'neural networks': 1}, {'neural networks': 1, 'deep learning': 1, 'Big Data': 1, 'artificial intelligence': 1}, {'Hadoop': 1, 'Java': 1, 'MapReduce': 1, 'Big Data': 1}, {'statistics': 1, 'R': 1, 'statsmodels': 1}, {'C++': 1, 'deep learning': 1, 'artificial intelligence': 1, 'probability': 1}, {'pandas': 1, 'R': 1, 'Python': 1}, {'databases': 1, 'HBase': 1, 'Postgres': 1, 'MySQL': 1,

array([[1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
        0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.,

## Коллаборативная фильтрация на основе пользователей
User-Based Collaborative Filtering

Идея - для пользователя найти других пользователей, которые на него похожи, и предложить те темы, которые они предпочитают

<img src="cfuser.png">

Как посчитать похожесть пользователей?

* Косинусная мера - косинус угла между векторами рейтингов. Если два вектора равнонаправлены, их косинусный коэффициент равен 1. Если направление противоположно, то -1.

$$ \text{similarity}=\cos(\theta )={\mathbf {A} \cdot \mathbf {B}  \over \|\mathbf {A} \|_{2}\|\mathbf {B} \|_{2}}={\frac {\sum \limits _{i=1}^{n}{A_{i}B_{i}}}{{\sqrt {\sum \limits _{i=1}^{n}{A_{i}^{2}}}}{\sqrt {\sum \limits _{i=1}^{n}{B_{i}^{2}}}}}} $$

В нашем случае присутствуют все "оценки" - пользователь интересуется темой (1) или не интересуется (0).  
В случае отсуствия оценок - брать пересечение предметов пользователей

* Коэффициент Пирсона

Пусть даны две выборки $x^m=\left( x_1, \cdots ,x_m  \right)$,  $y^m=\left( y_1, \cdots ,y_m  \right)$

$$r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}} = \frac {cov(x,y)}{\sqrt{s_x^2 s_y^2}}$$

где $\bar{x}$, $\bar{y}$ – выборочные средние $x^m$ и $y^m$, $s_x^2$,  $s_y^2$ – выборочные дисперсии, $r_{xy} \in \left[-1,1\right]$.


In [18]:
#
# User-Based Collaborative Filtering
#

user_similarities = cosine_similarity(user_interest_matrix, user_interest_matrix)
# схожесть первого пользователя с остальными
user_similarities[0]

array([1.        , 0.3380617 , 0.        , 0.        , 0.        ,
       0.15430335, 0.        , 0.        , 0.18898224, 0.56694671,
       0.        , 0.        , 0.        , 0.16903085, 0.        ])

Простейший способ рекомендации - просуммировать коэффициенты схожести пользователей по каждой теме (для нашего случая это подходит, так как оценки 1 или 0 и нам нужно ранжирование.

In [20]:
def most_similar_users_to(user_id):
    user = user_similarities[user_id]
    
    pairs = [(other_user_id, similarity)                      # find other
             for other_user_id, similarity in enumerate(user) # users with
             if user_id != other_user_id and similarity > 0]  # nonzero similarity

    return sorted(pairs, key=lambda x: x[1], reverse=True)    # sort them most similar first 


def user_based_suggestions(user_id, include_current_interests=False):
    # sum up the similarities
    suggestions = defaultdict(float)
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity

    # convert them to a sorted list
    suggestions = sorted(suggestions.items(),
                         key=lambda weight: weight[1],
                         reverse=True)

    # and (maybe) exclude already-interests
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight) 
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]
    
print("User based similarity")
print("most similar to 0")
print(users_interests[0])
print(users_interests[9])
print(most_similar_users_to(0))

print("Suggestions for 0")
print(user_based_suggestions(0))

User based similarity
most similar to 0
['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra']
['Hadoop', 'Java', 'MapReduce', 'Big Data']
[(9, 0.5669467095138407), (1, 0.3380617018914066), (8, 0.1889822365046136), (13, 0.1690308509457033), (5, 0.1543033499620919)]
Suggestions for 0
[('MapReduce', 0.5669467095138407), ('MongoDB', 0.50709255283711), ('Postgres', 0.50709255283711), ('NoSQL', 0.3380617018914066), ('neural networks', 0.1889822365046136), ('deep learning', 0.1889822365046136), ('artificial intelligence', 0.1889822365046136), ('databases', 0.1690308509457033), ('MySQL', 0.1690308509457033), ('Python', 0.1543033499620919), ('R', 0.1543033499620919), ('C++', 0.1543033499620919), ('Haskell', 0.1543033499620919), ('programming languages', 0.1543033499620919)]


Если нужно предсказать рейтинг (u - пользователь, i - предмет):

$$\hat{r}_{u,i} = \bar{r_i} + \frac{\sum_v s_{u,v} (r_{v, i} -  \bar{r_v})}{\sum_v |s_{u,v}|}$$

#### Замечения:

1. Пользователи часто ставят пессимистичные или оптимистичные оценки, поэтому считаем $\bar{r_u}$ - средняя оценка пользователя, и делаем поправку

<img src='star_ratings.png'>

2. Искать подробную информацию можно по "GroupLens algorithm"

3. Можно не суммировать по всем пользователям, а ограничиться ближайшими соседями

4. При росте размерности похожие пользователи перестают на самом деле быть похожими (проклятие размерности) и рекомендация работает плохо

5. Естественно предположить, что продукты, которые любят или не любят практически все пользователи, не слишком полезны в определении ближайшего соседа. Поэтому естественно взвесить продукты по тому, как часто их уже оценивали пользователи; такая метрика называется iuf – inverse user frequency, обратная частота пользователей

## Коллаборативная фильтрация по схожести предметов
Item-Based Collaborative Filtering

Будем считать сходство непосредственно между темами. После этого можно рекомендовать пользователю темы, похожие на те, которые его уже интересуют.

<img src="content.png">

In [13]:
#
# Item-Based Collaborative Filtering
#

interest_user_matrix = user_interest_matrix.T

interest_similarities = cosine_similarity(interest_user_matrix, interest_user_matrix)
interest_similarities[0]

array([1.        , 0.        , 0.40824829, 0.33333333, 0.81649658,
       0.        , 0.66666667, 0.        , 0.57735027, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.57735027, 0.57735027, 0.40824829, 0.        , 0.        ,
       0.40824829, 0.        , 0.        , 0.        , 0.40824829,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [14]:
interest_similarities

array([[1.        , 0.        , 0.40824829, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.40824829, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [22]:
def most_similar_interests_to(interest_id):
    similarities = interest_similarities[interest_id]
    pairs = [(unique_interests[other_interest_id], similarity)
             for other_interest_id, similarity in enumerate(similarities)
             if interest_id != other_interest_id and similarity > 0]
    return sorted(pairs,
                  key=lambda similarity: similarity[1],
                  reverse=True)

def item_based_suggestions(user_id, include_current_interests=False):
    suggestions = defaultdict(float)
    user_interest_vector = user_interest_matrix[user_id]
    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            similar_interests = most_similar_interests_to(interest_id)
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity

    suggestions = sorted(suggestions.items(),
                         key=lambda similarity: similarity[1],
                         reverse=True)

    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight) 
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]


print("Item based similarity")
print("most similar to 'Big Data'")
print(most_similar_interests_to(0))

print("suggestions for user 0")
print(item_based_suggestions(0))

Item based similarity
most similar to 'Big Data'
[('Hadoop', 0.816496580927726), ('Java', 0.6666666666666669), ('MapReduce', 0.5773502691896258), ('Spark', 0.5773502691896258), ('Storm', 0.5773502691896258), ('Cassandra', 0.408248290463863), ('artificial intelligence', 0.408248290463863), ('deep learning', 0.408248290463863), ('neural networks', 0.408248290463863), ('HBase', 0.3333333333333334)]
suggestions for user 0
[('MapReduce', 1.861807319565799), ('MongoDB', 1.3164965809277258), ('Postgres', 1.3164965809277258), ('NoSQL', 1.2844570503761732), ('MySQL', 0.5773502691896258), ('databases', 0.5773502691896258), ('Haskell', 0.5773502691896258), ('programming languages', 0.5773502691896258), ('artificial intelligence', 0.408248290463863), ('deep learning', 0.408248290463863), ('neural networks', 0.408248290463863), ('C++', 0.408248290463863), ('Python', 0.2886751345948129), ('R', 0.2886751345948129)]


#### Замечания
* Можно считать оффлайн, так как предметы появляются редко, оценка нескольких пользователей не влияет сильно на предмет
* Для новых предметов нужно каким-либо образом получить несколько оценок
* Эффект пузыря - никогда не получим рекомендацию новой, не известной нам ранее темы

#### Общие замечания
* Вычисления ресурсоемкие - все оценки нужно хранить в памяти
* Иногда пользователь кликнул, но ему не понравилось (!)

# https://grouplens.org/datasets/movielens/100k/

или https://www.kaggle.com/prajitdatta/movielens-100k-dataset

In [29]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader, Dataset, SVD, evaluate

In [30]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 y


Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\1/.surprise_data/ml-100k
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9338  0.9392  0.9422  0.9319  0.9421  0.9378  0.0043  
MAE (testset)     0.7358  0.7428  0.7441  0.7352  0.7387  0.7393  0.0036  
Fit time          3.34    3.35    3.37    3.33    3.36    3.35    0.02    
Test time         0.10    0.12    0.12    0.12    0.10    0.11    0.01    


{'test_rmse': array([0.9337621 , 0.93920189, 0.94220364, 0.93190383, 0.94207721]),
 'test_mae': array([0.73577947, 0.7427816 , 0.74408276, 0.73516385, 0.73867588]),
 'fit_time': (3.3380048274993896,
  3.354022741317749,
  3.372028112411499,
  3.3250298500061035,
  3.357023000717163),
 'test_time': (0.09999966621398926,
  0.12199997901916504,
  0.12099456787109375,
  0.12299251556396484,
  0.10200142860412598)}

In [31]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

C:\ProgramData\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9294
MAE:  0.7292
------------
Fold 2
RMSE: 0.9402
MAE:  0.7423
------------
Fold 3
RMSE: 0.9399
MAE:  0.7420
------------
Fold 4
RMSE: 0.9306
MAE:  0.7358
------------
Fold 5
RMSE: 0.9397
MAE:  0.7422
------------
------------
Mean RMSE: 0.9360
Mean MAE : 0.7383
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9294025822150249,
                             0.9401501180123344,
                             0.9399164452047318,
                             0.9305826158519822,
                             0.9397186155937872],
                            'mae': [0.7291808884208323,
                             0.7423092717731407,
                             0.7420277531139287,
                             0.7358404353513306,
                             0.742235510168149]})

In [32]:
trainset = data.build_full_trainset()
svd.train(trainset)

C:\ProgramData\Anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


In [33]:
svd.predict(1, 302, 5)

Prediction(uid=1, iid=302, r_ui=5, est=3.52986, details={'was_impossible': False})

# SVD

<img src='mf.png'>

Для любого пользователя можно предсказать оценки, которые он еще не ставил. Обучать модель нужно так, чтобы она давала наименьшие ошибки для уже известных моделей.

Предсказание - скалярное произведение вектора для пользователя и вектора для объекта.

При таком подходе необходимо обучить намного меньше параметров модели

SVD напрямую применить нельзя, так как очень много отсутствующих значений (точнее можно, как-то заполнив их, но получается не очень хорошее качество).  
Поэтому формулируется задача оптимизации по известным оценкам $K$:

$$ min_{p, q} \sum_{u,i \in K} (r_{ui} - q_i^T p_u)^2 + \lambda (||q_i||^2 + ||p_u||^2)$$

# Оценка качества

* Качество рейтингов
    * MAE, MSE
* Качество событий
    * F-score, ROC-AUC, PR-AUC
    * precision@k, recall@k
* Качество ранжирования
    * $DCG@k(u) = \sum\limits_{p=1}^k \frac{val(i,p)}{\log{(p+1)}}$
    * $nDCG@k(u) = \frac{DCG@k(u)}{\max{(DCG@k(u))}}$
   

1. https://habrahabr.ru/company/surfingbird/blog/139518/
2. https://d4datascience.wordpress.com/category/predictive-analytics/
3. Data Science from scratch
4. https://habrahabr.ru/company/yandex/blog/241455/
5. https://www.kaggle.com/rounakbanik/movie-recommender-systems/notebook
6. http://www.cs.ubbcluj.ro/~gabis/DocDiplome/SistemeDeRecomandare/Recommender_systems_handbook.pdf
7. https://www.coursera.org/specializations/recommender-systems
8. https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf
9. https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf
10. http://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD